```
Copyright 2024 Control-M Jupyter Notebooks Project Contributors

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
```

**Author:** Control-M Solution Engineering Team  
**E-Mail:** orchestrator@bmc.com  
**Version:** 1.0.0  
**License:** Apache 2.0  

**Warranty Disclaimer:** This software is provided "AS IS" without warranty of any kind. Use at your own risk.

# Control-M Configuration Data Collector

Collects servers, agents, hostgroups, connection profiles, and run-as users.

In [ ]:
import requests
import json
import os
import pandas as pd
import uuid
import platform
import socket
from datetime import datetime
from typing import Dict, List, Any, Union, Optional, Set
from IPython.display import display

In [ ]:
# Environment configuration
def load_env_file(filepath: str) -> Dict[str, str]:
    """Load environment variables from file"""
    env_vars: Dict[str, str] = {}
    try:
        with open(filepath, 'r') as f:
            for line in f:
                line = line.strip()
                if line and not line.startswith('#') and '=' in line:
                    key, value = line.split('=', 1)
                    env_vars[key.strip()] = value.strip().strip('"').strip("'")
        return env_vars
    except Exception:
        return {}

def save_response_to_file(data: Any, filename: str) -> None:
    """Save API response data to file"""
    try:
        os.makedirs('inventory', exist_ok=True)
        filepath = os.path.join('inventory', filename)
        with open(filepath, 'w') as f:
            json.dump(data, f, indent=2)
        print(f"  💾 Saved: {filepath}")
    except Exception as e:
        print(f"  ⚠️ Save error: {e}")

# Load environment - relative paths for same folder
env_files = ['.env', '../.env']
env_vars: Dict[str, str] = {}
for env_file in env_files:
    env_vars = load_env_file(env_file)
    if env_vars:
        print(f"📁 Loaded: {env_file}")
        break

CTM_HOST: Optional[str] = os.getenv("WERKSTATT_CTM_HOST") or env_vars.get("WERKSTATT_CTM_HOST")
CTM_PORT: Optional[str] = os.getenv("WERKSTATT_CTM_PORT") or env_vars.get("WERKSTATT_CTM_PORT")
AUTH_TOKEN: Optional[str] = os.getenv("WERKSTATT_CTM_AAPI") or env_vars.get("WERKSTATT_CTM_AAPI")

BASE_URL: Optional[str] = f"https://{CTM_HOST}:{CTM_PORT}/automation-api" if CTM_HOST and CTM_PORT else None
headers: Dict[str, str] = {"x-api-key": AUTH_TOKEN or "", "Content-Type": "application/json", "Accept": "application/json"}

print(f"✅ CTM server: {BASE_URL}")
print(f"✅ Token: {'SET' if AUTH_TOKEN else 'NOT SET'}")

In [ ]:
# Fast data collection - basic inventory only (no ping, no detailed params)
start_time = datetime.now()
detailed_agents: Dict[str, Dict[str, Any]] = {}
all_agents: List[Dict[str, Any]] = []
all_hostgroups: List[Dict[str, Any]] = []
all_runasusers: List[Dict[str, Any]] = []
total_agents_processed = 0

if not BASE_URL:
    print("❌ No BASE_URL configured. Please check environment variables.")
    servers = []
    cp_data = {}
else:
    # Get servers
    response = requests.get(f"{BASE_URL}/config/servers", headers=headers, verify=True)
    if response.status_code == 200:
        servers = response.json()
        save_response_to_file(servers, 'servers.json')
        print(f"✅ Servers: {len(servers)}")
    else:
        servers = []
        print(f"❌ Servers error: {response.status_code}")
    
    # Get connection profiles
    cp_response = requests.get(f"{BASE_URL}/deploy/connectionprofiles/centralized", headers=headers, verify=True)
    if cp_response.status_code == 200:
        cp_data = cp_response.json()
        save_response_to_file(cp_data, 'connection_profiles.json')
        print(f"✅ Connection profiles: {len(cp_data)}")
    else:
        cp_data = {}
        print(f"❌ Connection profiles error: {cp_response.status_code}")

# Process each server
for server in servers:
    server_name = server['name']
    print(f"\n🔍 Server: {server_name}")
    
    try:
        # Get agents - basic info only (fast)
        if BASE_URL:
            agents_response = requests.get(f"{BASE_URL}/config/server/{server_name}/agents", headers=headers, verify=True)
            if agents_response.status_code == 200:
                agents_data = agents_response.json()
                save_response_to_file(agents_data, f'agents_{server_name}.json')
                
                # Handle different agent data formats
                if isinstance(agents_data, dict) and 'agents' in agents_data:
                    agents_list = agents_data['agents']
                elif isinstance(agents_data, list):
                    agents_list = agents_data
                else:
                    agents_list = []
                    
                # Process each agent quickly - basic info only
                for agent in agents_list:
                    total_agents_processed += 1
                    
                    try:
                        # Extract agent name and basic info from agents list
                        if isinstance(agent, dict):
                            agent['server'] = server_name
                            all_agents.append(agent)
                            agent_name = agent.get('nodeid', str(agent))
                            agent_os = agent.get('operatingSystem', '')
                            agent_version = agent.get('version', '')
                            agent_tag = agent.get('tag', '')
                            agent_status = agent.get('status', 'Unknown')
                            agent_hostgroups = agent.get('hostgroups', [])
                            agent_type = agent.get('type', 'Agent')
                        elif isinstance(agent, str):
                            agent_name = agent
                            agent_os = agent_version = agent_tag = agent_status = agent_type = ''
                            agent_hostgroups = []
                            all_agents.append({'nodeid': agent_name, 'server': server_name})
                        else:
                            agent_name = str(agent)
                            agent_os = agent_version = agent_tag = agent_status = agent_type = ''
                            agent_hostgroups = []
                            all_agents.append({'nodeid': agent_name, 'server': server_name})
                        
                        if not agent_name:
                            continue
                        
                        # Create basic agent record (no ping, no detailed params)
                        detailed_agents[agent_name] = {
                            'name': agent_name,
                            'os': agent_os,
                            'version': agent_version,
                            'status': agent_status,
                            'tag': agent_tag,
                            'type': agent_type,
                            'server': server_name,
                            'hostgroups': agent_hostgroups
                        }
                        
                        print(f"    ✅ {agent_name}: {agent_status}")
                        
                    except Exception as agent_error:
                        print(f"    ❌ Agent error: {str(agent_error)}")
                        continue
                        
                print(f"  ✅ Processed {len(agents_list)} agents")
            else:
                print(f"  ❌ Agents request failed: {agents_response.status_code}")
        
        # Get hostgroups
        try:
            if BASE_URL:
                hg_response = requests.get(f"{BASE_URL}/config/server/{server_name}/hostgroups/agents", headers=headers, verify=True)
                if hg_response.status_code == 200:
                    hg_data = hg_response.json()
                    save_response_to_file(hg_data, f'hostgroups_{server_name}.json')
                    if isinstance(hg_data, list):
                        for hg in hg_data:
                            if isinstance(hg, dict):
                                hg['server'] = server_name
                                all_hostgroups.append(hg)
                    print(f"  ✅ Hostgroups: {len(hg_data) if isinstance(hg_data, list) else 0}")
        except Exception as hg_error:
            print(f"  ❌ Hostgroups: {hg_error}")
        
        # Get run as users
        try:
            if BASE_URL:
                ru_response = requests.get(f"{BASE_URL}/config/server/{server_name}/runasusers", headers=headers, verify=True)
                if ru_response.status_code == 200:
                    ru_data = ru_response.json()
                    save_response_to_file(ru_data, f'runasusers_{server_name}.json')
                    if isinstance(ru_data, list):
                        for ru in ru_data:
                            if isinstance(ru, dict):
                                ru['server'] = server_name
                                all_runasusers.append(ru)
                            else:
                                all_runasusers.append({'name': str(ru), 'server': server_name})
                    print(f"  ✅ Run as users: {len(ru_data) if isinstance(ru_data, list) else 0}")
        except Exception as ru_error:
            print(f"  ❌ Run as users: {ru_error}")
            
    except Exception as server_error:
        print(f"  ❌ Server {server_name}: {server_error}")
        continue

# Create dataframes
df_servers = pd.DataFrame(servers)
df_agents = pd.DataFrame(all_agents) if all_agents else pd.DataFrame()
df_hostgroups = pd.DataFrame(all_hostgroups) if all_hostgroups else pd.DataFrame()
df_connection_profiles = pd.DataFrame([{'profile_name': k, **v} for k, v in cp_data.items()]) if cp_data else pd.DataFrame()
df_runasusers = pd.DataFrame(all_runasusers) if all_runasusers else pd.DataFrame()

end_time = datetime.now()
duration = int((end_time - start_time).total_seconds())

print(f"\n📋 Summary:")
print(f"  📊 Servers: {len(df_servers)}")
print(f"  🤖 Agents: {len(df_agents)} (detailed: {len(detailed_agents)}, processed: {total_agents_processed})")
print(f"  👥 Hostgroups: {len(df_hostgroups)}")
print(f"  🔗 Connection profiles: {len(df_connection_profiles)}")
print(f"  👤 Run as users: {len(df_runasusers)}")
print(f"  ⚡ Duration: {duration} seconds (FAST - no ping/params)")

# Print list of all agents found
print(f"\n📋 All Agents Found:")
for agent_name, agent_info in detailed_agents.items():
    status = agent_info.get('status', 'Unknown')
    server = agent_info.get('server', '')
    print(f"  ✅ {agent_name} ({server}) - {status}")

In [ ]:
# Create comprehensive inventory file
destinations: Dict[str, Set[str]] = {
    'SFTP': set(), 'AS2': set(), 'S3': set(), 'GCS': set(), 
    'Azure': set(), 'SharePoint': set(), 'Local': set()
}

for profile_name, profile_data in cp_data.items():
    profile_type = profile_data.get('Type', '')
    if 'SFTP' in profile_type:
        hostname = profile_data.get('HostName')
        if hostname:
            destinations['SFTP'].add(hostname)
    elif 'AS2' in profile_type:
        hostname = profile_data.get('HostName')
        if hostname:
            destinations['AS2'].add(hostname)
    elif 'S3' in profile_type:
        region = profile_data.get('Region', '')
        access_key = profile_data.get('AccessKey', '')
        if region and access_key:
            destinations['S3'].add(f"Amazon:{region}:{access_key}")
    elif 'GCS' in profile_type:
        key_file = profile_data.get('GCSServiceAccountKeyFileName', '')
        if key_file:
            destinations['GCS'].add(key_file)
    elif 'Azure' in profile_type:
        account_name = profile_data.get('AzureAccountName', '')
        storage_type = profile_data.get('AzureStorageType', '')
        if account_name and storage_type:
            destinations['Azure'].add(f"SharedKey:{storage_type}:{account_name}")
    elif 'SharePoint' in profile_type:
        endpoint = profile_data.get('SharePointEndpoint', '')
        if endpoint:
            destinations['SharePoint'].add(endpoint)
    elif 'Local' in profile_type:
        user = profile_data.get('User', '')
        if user:
            destinations['Local'].add(user)

# Create inventory structure
inventory: Dict[str, Any] = {
    'uuid': str(uuid.uuid4()),
    'date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'platform': platform.system(),
    'hostname': socket.gethostname(),
    'EM': BASE_URL,
    'login': True,
    'duration': duration,
    'CCP': cp_data,
    'Servers': {},
    'Agents': detailed_agents,
    'HostGroups': {},
    'Lists': {
        'Destinations': {k: ', '.join(sorted(v)) for k, v in destinations.items() if v},
        'Profiles': ', '.join(sorted(cp_data.keys())),
        'Agents': ', '.join(sorted(detailed_agents.keys()))
    }
}

# Add servers
for server in servers:
    inventory['Servers'][server['name']] = {
        'name': server['name'],
        'host': server.get('host', server['name']),
        'state': server.get('state', 'Unknown'),
        'status': server.get('state', '').lower() == 'up'
    }

# Add hostgroups
hostgroup_dict: Dict[str, Dict[str, str]] = {}
for hg in all_hostgroups:
    hg_name = hg.get('hostgroup', hg.get('name', ''))
    if hg_name:
        agents_list = []
        if 'agentslist' in hg:
            agents_list = [agent.get('host', '') for agent in hg['agentslist'] if agent.get('host')]
        hostgroup_dict[hg_name] = {
            'name': hg_name,
            'members': ', '.join(agents_list) if agents_list else 'None'
        }

inventory['HostGroups'] = hostgroup_dict

# Save inventory file in root folder
inventory_path = 'ctm-inventory.json'
with open(inventory_path, 'w') as f:
    json.dump(inventory, f, indent=4)

print(f"\n✅ Inventory created: {inventory_path}")
print(f"📊 Summary:")
print(f"  UUID: {inventory['uuid']}")
print(f"  Servers: {len(inventory['Servers'])}")
print(f"  Agents: {len(inventory['Agents'])}")
print(f"  Hostgroups: {len(inventory['HostGroups'])}")
print(f"  Connection Profiles: {len(inventory['CCP'])}")
print(f"  Destinations: {sum(len(v.split(', ')) if v else 0 for v in inventory['Lists']['Destinations'].values())}")

In [ ]:
# Display data
print("🔗 Connection Profiles:")
display(df_connection_profiles)

In [ ]:
# Export to CSV - save in inventory folder to keep root clean
os.makedirs('inventory', exist_ok=True)
for name, df in [('servers', df_servers), ('agents', df_agents), ('hostgroups', df_hostgroups), 
                 ('connection_profiles', df_connection_profiles), ('runasusers', df_runasusers)]:
    if not df.empty:
        csv_path = os.path.join('inventory', f'ctm_{name}.csv')
        df.to_csv(csv_path, index=False)
        print(f"✅ Exported: {csv_path}")